### __Notebook to verify that looking at temperature amplitude ratio is interesting for a diffusive problem in the monodirectional case__ ###


In [ ]:
#Necessary imports

import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk 
from pyheatmy import * 
from pyheatmy.time_series_multiperiodic import time_series_multiperiodic

from sklearn import linear_model
from pyheatmy.core import *
import scipy

In [ ]:
"""Conditions limites"""
# Température de la rivière
T_riv_year_amp = 6 # °C, représente l'amplitude de variation de température annuelle
T_riv_offset = 12 + ZERO_CELSIUS # °C, représente l'offset commun de nos signaux de température
T_riv_day_amp = 1 # °C, représente l'amplitude de variation de température journalière
# Température de l'aquifère
T_aq_amp = 0
T_aq_offset = 12 + ZERO_CELSIUS
P_T_aq = -9999 # à mettre dans le init

P_T_riv_year = NDAYINYEAR * NSECINDAY # Période annuelle en mois
P_T_riv_day = NSECINDAY # Période journalière, en heures
t_debut = (2024, 4, 15, 8, 0, 0)  # (year, month, day, hour, minute, second)
t_fin = (2024, 4, 20, 8, 0, 0)
dt = int(NSECINHOUR / 4)  #On se place dans le cas d'un point de mesure toutes les quinze minutes (à moduler en fonction de l'intervale temporel considéré)

moinslog10IntrinK = 11
lambda_s = 2 # test cas purement advectif
rhos_cs = 4e6
range_of_minus_log_10_K = [10,11,12,13,14,15]  # The values we want to test

n = 0.1  # porosité
nb_cells=100 # nombre de cellules, une tous les 5 centimètres 
depth_sensors = [.1, .2, .3, .4]
Zbottom = 1
river_bed = 1.  # profondeur de la colonne en mètres
last_cell = int(9/10 * nb_cells)  # on écarte les dernières valeurs pour éviter les effets de bord
depth_cells = np.linspace(0, river_bed, nb_cells)

# Pression différentielle
dH_amp = 0
dH_offset = - 0.1  # m
P_dh = -9999 #14*24*4*dt

# Bruit de mesure
sigma_meas_P = 0.001
sigma_meas_T = 0.1

name ="Couche en sable"

In [ ]:
# Instanciation de time_series_multi_periodic
T_riv = time_series_multiperiodic("multi_periodic")

# On regarde des variations de température sur une année, on définit une période journalière, et une période annuelle

"""Reprise du code de dmo_genData pour créer un objet synthetic_MOLONARI"""
# un dictionnaire qui facilite le paramétrage avec des variables globales définies plus haut
time_series_dict_user1 = {
    "offset":.0,
    "depth_sensors":depth_sensors,
	"param_time_dates": [t_debut, t_fin, dt], 
    "param_dH_signal": [dH_amp, P_dh, dH_offset], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"param_T_riv_signal": [T_riv_day_amp, P_T_riv_day, T_riv_offset],
    "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
    "sigma_meas_P": sigma_meas_P,
    "sigma_meas_T": sigma_meas_T, #float
    "verbose": False
}

#On utilise le jeu de date précédent pour créer un signal de température multipériodique 
emu_observ_test_user1 = synthetic_MOLONARI.from_dict(time_series_dict_user1)


# signal multipériodique
T_riv.create_multiperiodic_signal([T_riv_year_amp, T_riv_day_amp], [[P_T_riv_year, 's'], [P_T_riv_day, 's']], emu_observ_test_user1._dates,
                                   offset=T_riv_offset, verbose = False)


T_riv.time_series_dict_user1 = time_series_dict_user1

#on génère un objet colonne à partir de l'objet emu_observ_test_user1
# modèle une couche
layers_list= layersListCreator([(name, Zbottom, moinslog10IntrinK, n, lambda_s, rhos_cs)])

T_riv.layers_list = layers_list

col_dict = {
	"river_bed": river_bed, 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": emu_observ_test_user1._molonariP_data,
    "T_measures": emu_observ_test_user1._molonariT_data,
    "sigma_meas_P": sigma_meas_P,
    "sigma_meas_T": sigma_meas_T,
}

T_riv.col_dict = col_dict
T_riv.nb_cells = nb_cells

T_riv.profil_temperature(verbose = True)


In [ ]:
#On vérifie que les températures ont bien été modifiées dans l'objet column (en particulier que la température à profondeur nulle est bien celle de la rivière)

plt.plot(emu_observ_test_user1._dates, T_riv.matrix[0,:])
plt.plot(emu_observ_test_user1._dates, col._temperatures[0,:])
plt.show()
print(f"La matrice de température a pour shape : {T_riv.matrix.shape}, abscisse = température aux 20 cellules, ordonnée = température à chaque pas de temps")
print(f"La matrice de température a pour shape : {col._temperatures.shape}, abscisse = température aux 20 cellules, ordonnée = température à chaque pas de temps")

In [ ]:
T_riv.plot_mosaic_pearson(range_of_minus_log_10_K)